In [ ]:
!pip install cohere

In [ ]:
import cohere

In [4]:
import pandas as pd

# Ruta del archivo
file_path = '../../0. Dataset/translated_dataset200.csv'

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(file_path)

# Mostrar las primeras filas del DataFrame
df

,Unnamed: 0,text,level,text_cleaned,label
0,0,"Vamos a ver una película, Anthony.\nEso suena ...",A1,"Vamos a ver una película, Anthony.\nEso suena ...",0
1,1,Todos los años vamos a Florida. Nos gusta ir a...,A1,Todos los años vamos a Florida. Nos gusta ir a...,0
2,2,Paula tiene cáncer. Le quedan seis meses de vi...,A1,Paula tiene cáncer. Le quedan seis meses de vi...,0
3,3,Hay una gran rotura aquí.\n¡Pero hay cinta en ...,A1,Hay una gran rotura aquí.\n¡Pero hay cinta en ...,0
4,4,Patty tiene una muñeca para Navidad. Es su muñ...,A1,Patty tiene una muñeca para Navidad. Es su muñ...,0
...,...,...,...,...,...
1195,1195,El rechazo anarquista de la autoridad tiene ap...,C2,El rechazo anarquista de la autoridad tiene ap...,5
1196,1196,Nadie sabe cuál fue el primero: la crisis econ...,C2,Nadie sabe cuál fue el primero: la crisis econ...,5
1197,1197,"Durante casi 100 años, la policía y las agenci...",C2,"Durante casi 100 años, la policía y las agenci...",5
1198,1198,El ataque mortal en Kabul contra los fieles ch...,C2,El ataque mortal en Kabul contra los fieles ch...,5


In [ ]:
co = cohere.Client("API_KEY_COHERE") # Get your API key: https://dashboard.cohere.com/api-keys

In [ ]:
prompt = lambda label, text:  f"""

A continuación, te proporcionaré un texto en español y te pediré que lo modifiques para diferentes niveles de competencia lingüística
(A1, A2, B1, B2, C1 y C2), concretamente: {label}. El objetivo es que adaptes el texto según el nivel de dificultad, modificando el
vocabulario y las estructuras gramaticales para que se ajusten a cada nivel, pero manteniendo el mismo mensaje central. Solo responde
con la version del texto modificada para dicho nivel. No incluyas ninguna introducción, título, explicación o comentario. Solamente dame
el texto adaptado.

Aquí está el texto:
{text}
"""

In [ ]:
# Extract n texts
import random
n = 240
lp = int(n/6)

indexes = []
for _ in range(n):
  indexes.append(random.randint(0, 1199))

texts = []
levels = []
for i in indexes:
  texts.append(df.text[i])
  levels.append(df.level[i])

levels_destined = ["A1"]*lp + ["A2"]*lp + ["B1"]*lp + ["B2"]*lp + ["C1"]*lp + ["C2"]*lp

In [ ]:
# Generate the response
import time
adapted_texts = []

for i in range(n):
  response = co.chat(message=prompt(levels_destined[i], texts[i]))
  adapted_texts.append(response.text)
  print(i)
  if(i!=0 and i%9==0): # Trial Key: only 10 API request per minute.
    time.sleep(60)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239


In [11]:
# Load Classifier model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Ruta de la carpeta donde está tu modelo
model_path = "../..//1. Clasificador (Entrenamiento)/Clasificador_200_Bert"

# Carga el tokenizador y el modelo desde la carpeta
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

/home/alexis/Documentos/Adaptacion-de-Textos-para-el-Aprendizaje-de-Lenguas-Castellanas/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Usar clasificador para ver niveles
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

adapted_texts_levels = []

for text in adapted_texts:
    # Tokenización del texto
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Inferencia del modeloz
    with torch.no_grad():
        outputs = model(**inputs)

    # Obtención de los logits
    logits = outputs.logits

    # Convertir logits en probabilidades usando softmax
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    # Obtener el índice de la clase con la mayor probabilidad
    predicted_class = torch.argmax(probabilities, dim=-1).item()

    # Mapear el índice de la clase a su respectivo nivel del MCER
    # (asumiendo que el modelo está entrenado para devolver clases en este orden: A1, A2, B1, B2, C1, C2)
    levels = ["A1", "A2", "B1", "B2", "C1", "C2"]
    predicted_level = levels[predicted_class]

    # Almacenar el nivel predicho
    adapted_texts_levels.append(predicted_level)

In [ ]:
# Save texts DRIVE
import numpy
texts = numpy.array(texts)
adapted_texts = numpy.array(adapted_texts)
levels_destined = numpy.array(levels_destined)
adapted_texts_levels = numpy.array(adapted_texts_levels)

numpy.save("/CohereFiles/texts5", texts)
numpy.save("/CohereFiles/adapted_texts5", adapted_texts)
numpy.save("/CohereFiles/levels_destined5", levels_destined)
numpy.save("/CohereFiles/adapted_texts_levels5", adapted_texts_levels)

In [ ]:
# Load texts DRIVE
texts = numpy.load("/CohereFiles/texts5.npy")
adapted_texts = numpy.load("/CohereFiles/adapted_texts5.npy")
levels_destined = numpy.load("/CohereFiles/levels_destined5.npy")
adapted_texts_levels = numpy.load("/CohereFiles/adapted_texts_levels5.npy")

In [ ]:
# Calculate percentaje of accuracy
n = len(texts)
level_to_number = {
    "A1": 0,
    "A2": 1,
    "B1": 2,
    "B2": 3,
    "C1": 4,
    "C2": 5
}

counter_exact = 0
counter_aprox = 0
for i in range(n):
  level_diff = abs(level_to_number[levels_destined[i]] - level_to_number[adapted_texts_levels[i]])
  if(level_diff == 0):
    counter_exact += 1
    counter_aprox += 1
  elif(level_diff == 1):
    counter_aprox += 0.5

percentage_exact = counter_exact/n
percentage_aprox = counter_aprox/n
print("Porcentaje de precisión (Exacto): ", percentage_exact)
print("Porcentaje de precisión (Aproximado): ", percentage_aprox)

Porcentaje de precisión (Exacto):  0.31
Porcentaje de precisión (Aproximado):  0.5220833333333333
1200


In [ ]:
# Extra Step - Make big sample
import numpy
# Load texts DRIVE
texts1 = numpy.load("/CohereFiles/texts1.npy")
adapted_texts1 = numpy.load("/CohereFiles/adapted_texts1.npy")
levels_destined1 = numpy.load("/CohereFiles/levels_destined1.npy")
adapted_texts_levels1 = numpy.load("/CohereFiles/adapted_texts_levels1.npy")
# Load texts DRIVE
texts2 = numpy.load("/CohereFiles/texts2.npy")
adapted_texts2 = numpy.load("/CohereFiles/adapted_texts2.npy")
levels_destined2 = numpy.load("/CohereFiles/levels_destined2.npy")
adapted_texts_levels2 = numpy.load("/CohereFiles/adapted_texts_levels2.npy")
# Load texts DRIVE
texts3 = numpy.load("/CohereFiles/texts3.npy")
adapted_texts3 = numpy.load("/CohereFiles/adapted_texts3.npy")
levels_destined3 = numpy.load("/CohereFiles/levels_destined3.npy")
adapted_texts_levels3 = numpy.load("/CohereFiles/adapted_texts_levels3.npy")
# Load texts DRIVE
texts4 = numpy.load("/CohereFiles/texts4.npy")
adapted_texts4 = numpy.load("/CohereFiles/adapted_texts4.npy")
levels_destined4 = numpy.load("/CohereFiles/levels_destined4.npy")
adapted_texts_levels4 = numpy.load("/CohereFiles/adapted_texts_levels4.npy")
# Load texts DRIVE
texts5 = numpy.load("/CohereFiles/texts5.npy")
adapted_texts5 = numpy.load("/CohereFiles/adapted_texts5.npy")
levels_destined5 = numpy.load("/CohereFiles/levels_destined5.npy")
adapted_texts_levels5 = numpy.load("/CohereFiles/adapted_texts_levels5.npy")


In [ ]:
texts = numpy.concatenate((texts1, texts2, texts3, texts4, texts5))
adapted_texts = numpy.concatenate((adapted_texts1, adapted_texts2, adapted_texts3, adapted_texts4, adapted_texts5))
levels_destined = numpy.concatenate((levels_destined1, levels_destined2, levels_destined3, levels_destined4, levels_destined5))
adapted_texts_levels = numpy.concatenate((adapted_texts_levels1, adapted_texts_levels2, adapted_texts_levels3, adapted_texts_levels4, adapted_texts_levels5))

In [ ]:
numpy.save("/CohereFiles/texts", texts)
numpy.save("/CohereFiles/adapted_texts", adapted_texts)
numpy.save("/CohereFiles/levels_destined", levels_destined)
numpy.save("/CohereFiles/adapted_texts_levels", adapted_texts_levels)